In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [26]:
torch.manual_seed(1) # 똑같은 결과를 보장하기 위해서 seed를 고정시킨다.

In [27]:
z = torch.FloatTensor([1, 2, 3]) #max의 결과 같은 (0, 0, 1)이 될 것 이다. 

In [28]:
hypothesis = F.softmax(z, dim=0) #softmax 3가지를 합쳐서 확률 값으로 나타낸다. 
print(hypothesis)#시그마 e^1/e^1+e^2+e^3=0.0900

tensor([0.0900, 0.2447, 0.6652])


In [29]:
hypothesis.sum()
#cross entropy loss = 1/n시그마 -ylog(y^h)

tensor(1.)

In [30]:
#크로그 엔트로피: 두개의 확률분포가 얼마나 비슷한지를 나타내는 함수

In [31]:
z = torch.rand(3, 5, requires_grad= True)
hypothesis = F.softmax(z, dim=1)
print(hypothesis)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward0>)


In [32]:
y = torch.randint(5, (3,)).long()
print(y)

tensor([0, 2, 1])


In [33]:
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_( 1, y.unsqueeze(1),1)#_가 있기 떄문에 y_one_hot값이 교체되어버린다.

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [34]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.4689, grad_fn=<MeanBackward0>)


In [35]:
torch.log(F.softmax(z,dim=1))

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward0>)

In [36]:
F.log_softmax(z, dim=1)

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward0>)

In [37]:
F.nll_loss(F.log_softmax(z, dim=1), y) #negative Log Likelihood

tensor(1.4689, grad_fn=<NllLossBackward0>)

In [38]:
F.cross_entropy(z, y)

tensor(1.4689, grad_fn=<NllLossBackward0>)

In [39]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]] #x_train = (m,4)
y_train = [2, 2, 2, 1, 1, 1, 0, 0] #y_train = (m,)
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [40]:
W = torch.zeros((4, 3), requires_grad=True)#클래스가 3개 이다. 
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1)
    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
    
    optimizer.zero_grad() # grad를 구한게 있으면 0으로 처리
    cost.backward() #cost에다가 backpropagation 수행, 미분
    optimizer.step() #cost 값을 minimize하는 방향으로 업데이트를 한다.
    
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs,  cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


In [17]:
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    z = x_train.matmul(W) + b
    cost = F.cross_entropy(z, y_train)
    
    optimizer.zero_grad() # grad를 구한게 있으면 0으로 처리
    cost.backward() #cost에다가 backpropagation 수행
    optimizer.step() #cost 값을 minimize하는 방향으로 업데이트를 한다.
    
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs,  cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


In [18]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3)
        
    def forward(self, x):
        return self.linear(x)

In [19]:
model = SoftmaxClassifierModel()

In [20]:
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    prediction = model(x_train)
    cost = F.cross_entropy(prediction, y_train)
    
    optimizer.zero_grad() # grad를 구한게 있으면 0으로 처리
    cost.backward() #cost에다가 backpropagation 수행
    optimizer.step() #cost 값을 minimize하는 방향으로 업데이트를 한다.
    
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs,  cost.item()
        ))

Epoch    0/1000 Cost: 1.849513
Epoch  100/1000 Cost: 0.689894
Epoch  200/1000 Cost: 0.609258
Epoch  300/1000 Cost: 0.551218
Epoch  400/1000 Cost: 0.500141
Epoch  500/1000 Cost: 0.451947
Epoch  600/1000 Cost: 0.405051
Epoch  700/1000 Cost: 0.358733
Epoch  800/1000 Cost: 0.312912
Epoch  900/1000 Cost: 0.269522
Epoch 1000/1000 Cost: 0.241922
